## `RSI & Bolinger Band & MACD`

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import talib as ta
from talib import MA_Type

#### Download Data

In [2]:
stock_symbol = "TLSA"
stock_data = yf.download(stock_symbol, period="max")

# Create a Date column
stock_data['Date'] = stock_data.index

# Drop the Date as index
stock_data.reset_index(drop=True, inplace=True)
stock_data.head(5)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Date
Ticker,TLSA,TLSA,TLSA,TLSA,TLSA,
0,1.670668,2.100840,1.670668,2.100840,87465,2018-11-20
1,2.118848,2.434974,1.838735,1.838735,38984,2018-11-21
2,1.960784,2.200880,1.958784,2.200880,14994,2018-11-23
3,1.676671,1.940776,1.552621,1.940776,24990,2018-11-26
4,1.792717,1.800720,1.700680,1.760704,21991,2018-11-27


#### Clean Data

In [3]:
dt_all = pd.date_range(start=stock_data.index[0],end=stock_data.index[-1])

dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(stock_data.index)]

dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

#### Plot

In [4]:
fig = go.Figure(data=[go.Candlestick(x=stock_data['Date'], open=stock_data['Open'],
                                     high=stock_data['High'], low=stock_data['Low'],
                                     close=stock_data['Close'])])

# Sets customized padding
fig.update_layout(margin=go.layout.Margin(r=10,b=10))

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')

fig.update_layout(title = stock_symbol + ' - CandleStick Chart',
                  xaxis_rangeslider_visible=False,
                  height=500, template='plotly_dark')

### RSI

In [ ]:
# RSI
stock_data['RSI'] = ta.RSI(stock_data.Close.values, 21)


# Construct a 2 x 1 Plotly figure
fig = make_subplots(row=2, cols=1, vertical_spacing=0.01, shared_xaxes=True)

# Plot RSI
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['RSI'], name='RSI'), row=1, col=1)

fig.add_hline(y=30,line_dash='dash', line_color='limegreen', line_width=1)
fig.add_hline(y=70, line_dash='dash', line_color='red', line_width=1)
fig.update_yaxes(title_text='RSI Score')

colors = ['#9C1F0B' if row['Open'] - row['Close'] >= 0
      else '#2B8308' for index, row in stock_data.iterrows()]

fig.add_trace(go.Bar(x=stock_data['Date'], y=stock_data['Volume'],
                     showlegend=False, marker_color=colors), row=2, col=1)

# Adds the range selector
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=1, label='1m', step='month', stepmode='backword'),
                dict(count=6, label='6m', step='month', stepmode='backword'),
                dict(count=1, label='YTD', step='year', stepmode='todate'),
                dict(count=1, label='1y', step='year', stempmode='backword'),
                dict(step='all')
            ]
        ),
    type='date'
    ),
    xaxis2_type='date'
)


# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector=dict(font=dict(color='black'))))

# Set customized padding
fig.update_layout(margin=go.layout.Margin(r=10, b=10))

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

fig['layout']['yaxis']['title']='RSI score'
fig['layout']['yaxis2']['title']='Volume'
fig['layout']['xaxis2']['title']='Date'

layout = go.Layout(template='plotly_dark', title=stock_symbol + ' - RSI', height=500, legend='Legends',
                   xaxis=dict(rangeselector=dict(font=dict(color='black'))))

fig.update_layout(layout)


### Bollinger Band

In [ ]:
stock_data['BU'], stock_data['BM'], stock_data['BL'] = ta.BBANDS(
    stock_data.Close, timeperiod=20, matype=MA_Type.EMA
)

fig = px.line(data_frame=stock_data, x = stock_data.index, y=['Close', 'BU', 'BM', 'BL'])

# Update y & x axis labels
fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')

fig.data[0].name = 'Price'

# Set customized padding
fig.update_layout(margin=go.layout.Margin(r=10, b=10))

layout = go.Layout(template='plotly_dark', title=stock_symbol + ' - Price, Bollinger Bands',
                   height=500, legend_title = 'Legend')

fig.update_layout(layout)

In [ ]:
# Drop Buy and Sell Column if they exist
stock_data.drop(['Buy', 'Sell'], inplace=True, axis=1, errors='ignore')

# Create DataFrame
df_buy  = stock_data.query('Low < BL')[['Date', 'Close']]
df_sell = stock_data.query('High > BL')[['Date', 'Close']]

# Round close values for both buy and sell
df_buy['Close'] = round(df_buy.Close.round())
df_sell['Close'] = round(df_sell.Close.round())

In [ ]:
# Plot

fig = go.Figure(data=[go.Candlestick(x=stock_data["Date"], open=stock_data["Open"],
                                     high=stock_data["High"],
                                     low=stock_data["Low"],
                                     close=stock_data["Close"],
                                     name="Candlestick")])

# Plot BU line graph; don't show legend
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data["BU"],
                         fill=None, mode="lines", showlegend=False))


# Plot BL line graph; don't show legend
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['BL'],
                         fill=None, mode='lines', showlegend=False))

# Plot Buy signals
fig.add_trace(go.Scatter(x=df_buy['Date'], y=df_buy["Close"], mode="markers",
                         marker=dict(symbol="diamond", size=7, line=dict(width=1)), name="Sell"))


fig.update_yaxes(title_text="Price")
fig.update_xaxes(title_text="Date")

fig.data[0].name="Price"
fig.update_layout(margin=go.layout.Margin(r=10, b=10))
layout = go.Layout(template="plotly_dark",
                   title=stock_symbol + " - Buy / Sell Signals", heights=500, 
                   xaxis_rangeslider_visible=False)

fig.update_layout(layout)


In [ ]:
# Calculate MACD values
# Empty Data Frame to collect MACD analysis reuslts

analysis = pd.DataFrame()
analysis['macd'], analysis['macdSignal'], analysis['macdHist'] = ta.MACD(stock_data.close,
                                                                         fastperiod=12,
                                                                         slowperiod=26,
                                                                         signalperiod=9)
fig = make_subplots(rows=2, cols=1)

# Candlestick chart for pricing
fig.append_trace(go.Candlestick(x=stock_data['Date'], open=stock_data['Open'],
                                high=stock_data['High'], low=stock_data['Low'],
                                close=stock_data['Close'], showlegend=False),
                                row=1, col=1)

# Fast Signal (%k)
fig.append_trace(go.Scatter(x=stock_data['Date'],
                            y=analysis['macd'],
                            line=dict(color="#C42836", widht=1),
                            name="MACD Line"), row = 2, col = 1)

# Slow signal (%d)
fig.append_trace(go.Scatter(x=stock_data['Date'], y=analysis['macdSignal'],
                            line=dict(color='limegreen', width=1),
                            name='Signal Line'), row=2, col=1)

# Colorize the histogram values
colors = np.where(analysis['macd'] < 0, '#EA071C', '#57F291')

# Plot the histogram
fig.append_trace(go.Bar(x=stock_data['Date'], y=analysis['macdHist'],
                        name='Histogram', maker_color=colors),
                        row=2, col=1)

fig['layout']['yaxis']['title']='Price'
fig['layout']['xaxis2']['title']='Date'

fig.data[0].name = 'Price'

# Set customized padding
fig.update_layout(margin=go.layout.Margin(r=10, b=10))

# Make it pritty
layout = go.Layout(template='plotly_dark', title=stock_symbol + ' - MACD Indicator', height=700,
                   xaxis_rangeslider_visible=False)

# Update options and show plot
fig.update_layout(layout)

fig.update_layout(legend=dict(yanchor="top", y=0.45, xanchor="left", x=1.01))
fig.show()